In [52]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [41]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510443 -0.7733844   0.5927712  ...  0.5792355   0.32683477
  -0.6508964 ]
 [-0.09361734 -0.18191512 -0.19230866 ... -0.0316581   0.30412546
  -0.26793587]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [42]:
df = pd.read_csv('/Users/kimdohyeon/건양대학교병원_바이오헬스/Biomedical_AI_Train/Mental_health_chatbot/wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [43]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [44]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [45]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [46]:
model.encode(df.loc[0, '유저'])

array([-4.80605811e-01, -2.94869095e-01,  4.37900126e-01, -6.40137315e-01,
        3.28669064e-02, -3.42647493e-01, -5.47481850e-02,  1.73054226e-02,
       -4.08220500e-01, -5.06033719e-01, -1.68733329e-01, -3.98677140e-01,
       -1.24776892e-01, -9.71543118e-02, -1.65286466e-01,  5.72591787e-03,
        6.13689944e-02, -1.91312209e-01,  2.53917038e-01, -5.85019648e-01,
       -2.84426302e-01, -2.32035205e-01, -3.27080488e-01,  6.72978684e-02,
       -1.65381607e-05, -4.72335756e-01, -3.60022068e-01,  2.91879326e-01,
       -6.63861096e-01, -3.10574263e-01,  5.79524875e-01, -3.11722755e-01,
        1.47701055e-02, -2.12172508e-01,  2.22058520e-01, -1.73828915e-01,
       -3.78458440e-01, -4.20397937e-01, -2.38219157e-01,  6.38706386e-02,
       -1.15304522e-01, -2.44564056e-01, -5.00228882e-01,  1.68355420e-01,
       -6.58359945e-01, -8.91942680e-01, -6.26956940e-01, -3.21965069e-01,
       -7.05357850e-01,  3.71447355e-01, -5.45803726e-01,  7.76294321e-02,
        1.09864153e-01,  

# 유저 대화내용 인코딩

In [47]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806058, -0.2948691, 0.43790013, -0.640137..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561569, -0.14506237, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66519976, -0.08126874, 1.0945566, 0.105792..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679061, 0.46520716, 0.5285069, -0.5076044..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277742, -0.3741391, 0.04053235, -0.86243..."


In [48]:
df.to_csv('wellness_datasets.csv', index=False)

# 간단한 챗봇

In [49]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [50]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806058, -0.2948691, 0.43790013, -0.640137...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561569, -0.14506237, 0.2949032, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66519976, -0.08126874, 1.0945566, 0.105792...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679061, 0.46520716, 0.5285069, -0.5076044...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277742, -0.3741391, 0.04053235, -0.86243...",0.315118


In [51]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

구분 증상/편두통
유사한 질문 요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇 답변 으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
유사도 0.8296288847923279
